In [ ]:
import os
import torch
import torchvision
import timm
import numpy
import PIL.Image
import faiss
import tqdm
import matplotlib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_resnet = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
])

transform_effnet = torchvision.transforms.Compose([
    torchvision.transforms.Resize((300, 300)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
])

resnet_model = torchvision.models.resnet50(pretrained=True)
resnet_model.fc = torch.nn.Identity()
resnet_model = resnet_model.to(device)
resnet_model.eval()

effnet_model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=0, global_pool='avg')
effnet_model = effnet_model.to(device)
effnet_model.eval()

def extract_features(model, transform, image_path):
    try:
        image = PIL.Image.open(image_path).convert("RGB")
    except Exception as e:
        raise RuntimeError(f"[Open error] {image_path}: {e}")
    
    try:
        tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = model(tensor)
            embedding = features.squeeze().detach().cpu().numpy().astype('float32')
            return embedding
    except Exception as e:
        raise RuntimeError(f"[Model error] {image_path}: {e}")

image_directory = "./flowers/train"
image_paths = [os.path.join(image_directory, f) for f in os.listdir(image_directory)
               if os.path.isfile(os.path.join(image_directory, f)) and f.lower().endswith(('.jpg', '.jpeg', '.png'))]

all_paths = []
resnet_embeddings = []
effnet_embeddings = []

for path in tqdm.tqdm(image_paths, desc="Extracting embeddings"):
    try:
        resnet_emb = extract_features(resnet_model, transform_resnet, path)
        effnet_emb = extract_features(effnet_model, transform_effnet, path)

        resnet_embeddings.append(resnet_emb)
        effnet_embeddings.append(effnet_emb)
        all_paths.append(path)
    except Exception as e:
        print(f"Error with {path}: {e}")

os.makedirs("./data", exist_ok=True)

resnet_embeddings = numpy.array(resnet_embeddings)
effnet_embeddings = numpy.array(effnet_embeddings)
all_paths = numpy.array(all_paths)

numpy.save("./data/resnet_embs.npy", resnet_embeddings)
numpy.save("./data/effnet_embs.npy", effnet_embeddings)
numpy.save("./data/paths.npy", all_paths)

if resnet_embeddings.shape[0] > 0:
    resnet_index = faiss.IndexFlatL2(resnet_embeddings.shape[1])
    resnet_index.add(resnet_embeddings)
    faiss.write_index(resnet_index, "./data/resnet.index")

if effnet_embeddings.shape[0] > 0:
    effnet_index = faiss.IndexFlatL2(effnet_embeddings.shape[1])
    effnet_index.add(effnet_embeddings)
    faiss.write_index(effnet_index, "./data/effnet.index")

def get_top5(image_path):
    resnet_query = extract_features(resnet_model, transform_resnet, image_path)
    effnet_query = extract_features(effnet_model, transform_effnet, image_path)

    distances_resnet, indices_resnet = resnet_index.search(numpy.expand_dims(resnet_query, axis=0), 5)
    distances_effnet, indices_effnet = effnet_index.search(numpy.expand_dims(effnet_query, axis=0), 5)

    top5_resnet = [(all_paths[i], distances_resnet[0][j]) for j, i in enumerate(indices_resnet[0])]
    top5_effnet = [(all_paths[i], distances_effnet[0][j]) for j, i in enumerate(indices_effnet[0])]

    return top5_resnet, top5_effnet

def show_comparison(query_path, resnet_top, effnet_top):
    figure, axes = matplotlib.pyplot.subplots(3, 6, figsize=(20, 10))

    image = PIL.Image.open(query_path)
    axes[0, 0].imshow(image)
    axes[0, 0].set_title("Query")
    axes[0, 0].axis("off")
    for i in range(1, 6):
        axes[0, i].axis("off")

    for i, (path, score) in enumerate(resnet_top):
        axes[1, i].imshow(PIL.Image.open(path))
        axes[1, i].set_title(f"ResNet {i+1}\n{score:.4f}")
        axes[1, i].axis("off")

    for i, (path, score) in enumerate(effnet_top):
        axes[2, i].imshow(PIL.Image.open(path))
        axes[2, i].set_title(f"EffNet {i+1}\n{score:.4f}")
        axes[2, i].axis("off")

    matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.show()


Number of training images found: 2762


Extracting embeddings:   1%|▏         | 37/2762 [00:00<00:07, 364.86it/s]

Error with ./flowers/train/8394286483_69fe04cc7f.jpg: [Model error] ./flowers/train/8394286483_69fe04cc7f.jpg: Numpy is not available
Error with ./flowers/train/480621885_4c8b50fa11_m.jpg: [Model error] ./flowers/train/480621885_4c8b50fa11_m.jpg: Numpy is not available
Error with ./flowers/train/14055574740_bb6a11e983_n.jpg: [Model error] ./flowers/train/14055574740_bb6a11e983_n.jpg: Numpy is not available
Error with ./flowers/train/26685647236_8211cb3e95_n.jpg: [Model error] ./flowers/train/26685647236_8211cb3e95_n.jpg: Numpy is not available
Error with ./flowers/train/6209630964_e8de48fe04_m.jpg: [Model error] ./flowers/train/6209630964_e8de48fe04_m.jpg: Numpy is not available
Error with ./flowers/train/16691277899_9433f39155_n.jpg: [Model error] ./flowers/train/16691277899_9433f39155_n.jpg: Numpy is not available
Error with ./flowers/train/4668543441_79040ca329_n.jpg: [Model error] ./flowers/train/4668543441_79040ca329_n.jpg: Numpy is not available
Error with ./flowers/train/3954167

Extracting embeddings:   4%|▍         | 109/2762 [00:00<00:07, 340.00it/s]

Error with ./flowers/train/14003401241_543535b385.jpg: [Model error] ./flowers/train/14003401241_543535b385.jpg: Numpy is not available
Error with ./flowers/train/7176736574_14446539cb_n.jpg: [Model error] ./flowers/train/7176736574_14446539cb_n.jpg: Numpy is not available
Error with ./flowers/train/5182167964_9d1a0be0b8_n.jpg: [Model error] ./flowers/train/5182167964_9d1a0be0b8_n.jpg: Numpy is not available
Error with ./flowers/train/34365073265_246c5c0561_n.jpg: [Model error] ./flowers/train/34365073265_246c5c0561_n.jpg: Numpy is not available
Error with ./flowers/train/34550871222_a9d795a2c0_n.jpg: [Model error] ./flowers/train/34550871222_a9d795a2c0_n.jpg: Numpy is not available
Error with ./flowers/train/4558562689_c8e2ab9f10.jpg: [Model error] ./flowers/train/4558562689_c8e2ab9f10.jpg: Numpy is not available
Error with ./flowers/train/17198868382_697b23c715_n.jpg: [Model error] ./flowers/train/17198868382_697b23c715_n.jpg: Numpy is not available
Error with ./flowers/train/2001380

...

Error with ./flowers/train/3894586562_5dbbdc4354_n.jpg: [Model error] ./flowers/train/3894586562_5dbbdc4354_n.jpg: Numpy is not available
Error with ./flowers/train/116343334_9cb4acdc57_n.jpg: [Model error] ./flowers/train/116343334_9cb4acdc57_n.jpg: Numpy is not available
Error with ./flowers/train/4669006062_6b3d260037_n.jpg: [Model error] ./flowers/train/4669006062_6b3d260037_n.jpg: Numpy is not available
Error with ./flowers/train/33874126263_3f6f965784_n.jpg: [Model error] ./flowers/train/33874126263_3f6f965784_n.jpg: Numpy is not available
Error with ./flowers/train/13979098015_e8c98fd34e_n.jpg: [Model error] ./flowers/train/13979098015_e8c98fd34e_n.jpg: Numpy is not available
Error with ./flowers/train/3334350831_f8755a2095_n.jpg: [Model error] ./flowers/train/3334350831_f8755a2095_n.jpg: Numpy is not available
Error with ./flowers/train/2828733818_1c1ed0089d_n.jpg: [Model error] ./flowers/train/2828733818_1c1ed0089d_n.jpg: Numpy is not available
Error with ./flowers/train/43607

Extracting embeddings:  99%|█████████▉| 2734/2762 [00:09<00:00, 313.74it/s]

Error with ./flowers/train/5795159787_ebb51a5e75.jpg: [Model error] ./flowers/train/5795159787_ebb51a5e75.jpg: Numpy is not available
Error with ./flowers/train/4503599544_3822e7d1be.jpg: [Model error] ./flowers/train/4503599544_3822e7d1be.jpg: Numpy is not available
Error with ./flowers/train/4858372040_52216eb0bd.jpg: [Model error] ./flowers/train/4858372040_52216eb0bd.jpg: Numpy is not available
Error with ./flowers/train/13903988248_22da33f341.jpg: [Model error] ./flowers/train/13903988248_22da33f341.jpg: Numpy is not available
Error with ./flowers/train/9200211647_be34ce978b.jpg: [Model error] ./flowers/train/9200211647_be34ce978b.jpg: Numpy is not available
Error with ./flowers/train/4764674741_82b8f93359_n.jpg: [Model error] ./flowers/train/4764674741_82b8f93359_n.jpg: Numpy is not available
Error with ./flowers/train/33851560384_d2e7d933db_n.jpg: [Model error] ./flowers/train/33851560384_d2e7d933db_n.jpg: Numpy is not available
Error with ./flowers/train/15240466871_ec45b65554_

Extracting embeddings: 100%|██████████| 2762/2762 [00:09<00:00, 300.84it/s]

Error with ./flowers/train/5172171681_5934378f08.jpg: [Model error] ./flowers/train/5172171681_5934378f08.jpg: Numpy is not available
Error with ./flowers/train/18250039435_7654bc11be_n.jpg: [Model error] ./flowers/train/18250039435_7654bc11be_n.jpg: Numpy is not available
Error with ./flowers/train/2365428551_39f83f10bf_n.jpg: [Model error] ./flowers/train/2365428551_39f83f10bf_n.jpg: Numpy is not available
Error with ./flowers/train/14886860069_b84665a073.jpg: [Model error] ./flowers/train/14886860069_b84665a073.jpg: Numpy is not available
Error with ./flowers/train/14928117202_139d2142cc_n.jpg: [Model error] ./flowers/train/14928117202_139d2142cc_n.jpg: Numpy is not available
Error with ./flowers/train/3311874685_7b9ef10f7e_m.jpg: [Model error] ./flowers/train/3311874685_7b9ef10f7e_m.jpg: Numpy is not available
Error with ./flowers/train/14970973709_968910640e_n.jpg: [Model error] ./flowers/train/14970973709_968910640e_n.jpg: Numpy is not available
Error with ./flowers/train/3459186